<a href="https://colab.research.google.com/github/ShuqairABD/MetaData-tools/blob/main/Metadata_image_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# !pip / libraries

In [ ]:
# !pip install xmp
# !pip install python-xmp-toolkit
# !pip install pyexiv2
# !pip install piexif
# !apt-get install -y exiftool
# !pip install iptcinfo3

In [7]:
import os
import shutil
from google.colab import files
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from time import sleep
from ipywidgets import interact_manual, Dropdown, Text
from ipywidgets import Text, Dropdown, Button, widgets
from iptcinfo3 import IPTCInfo
import piexif
from zipfile import ZipFile

# **functions**

In [ ]:
# Функция для добавления ключевого слова в метаданные XMP
def add_keyword_xmp(image_path, keyword):
    try:
        os.system(f'exiftool -XMP:Subject+=" {keyword}" "{image_path}"')
        print(f"Ключевое слово '{keyword}' добавлено к метаданным XMP {image_path}")
    except Exception as e:
        print(f"Ошибка при добавлении ключевого слова к метаданным XMP {image_path}: {str(e)}")


# Функция для добавления ключевых слов в метаданные EXIF
def add_keywords_to_image_exif(image_path, keywords):
    try:
        # Загрузка существующих метаданных EXIF
        exif_dict = piexif.load(image_path)

        # Извлечение существующих ключевых слов
        existing_keywords = exif_dict["Exif"].get(piexif.ExifIFD.UserComment, b"").decode("utf-8")
        existing_keywords_list = existing_keywords.split(",") if existing_keywords else []

        # Добавление новых ключевых слов к существующим
        existing_keywords_list.extend(keywords)

        # Объединение ключевых слов обратно в строку
        keyword_str = ",".join(existing_keywords_list)

        # Кодирование ключевых слов в байты и обновление метаданных EXIF
        exif_dict["Exif"][piexif.ExifIFD.UserComment] = keyword_str.encode('utf-8')

        # Преобразование метаданных EXIF обратно в двоичный формат
        exif_bytes = piexif.dump(exif_dict)

        # Вставка обновленных метаданных EXIF в файл изображения
        piexif.insert(exif_bytes, image_path)

        print(f"Ключевые слова '{keywords}' добавлены к метаданным EXIF.")
    except Exception as e:
        print(f"Произошла ошибка при добавлении ключевых слов к метаданным EXIF: {str(e)}")


# Функция для добавления ключевых слов в метаданные IPTC
def add_keywords_to_image_iptc(image_path, keywords):
    try:
        info = IPTCInfo(image_path)
        existing_keywords = info['keywords']
        for keyword in keywords:
            if keyword not in existing_keywords:
                existing_keywords.append(keyword)
        info['keywords'] = existing_keywords
        info.save()
        print(f"Ключевые слова '{keywords}' добавлены к метаданным IPTC.")
    except Exception as e:
        print(f"Произошла ошибка при добавлении ключевых слов к метаданным IPTC: {str(e)}")

# Функция для тегирования изображений в зависимости от выбранного метода
def tag_images(folder_path, method, keywords):
    tagged_images = []  # Список для хранения путей к тегированным изображениям

    try:
        if method == 'XMP':
            for filename in os.listdir(folder_path):
                if filename.endswith(('.jpg', '.jpeg', '.png')):
                    image_path = os.path.join(folder_path, filename)
                    add_keyword_xmp(image_path, keywords)
                    tagged_images.append(image_path)
        elif method == 'EXIF':
            for filename in os.listdir(folder_path):
                if filename.endswith(('.jpg', '.jpeg', '.png')):
                    image_path = os.path.join(folder_path, filename)
                    add_keywords_to_image_exif(image_path, keywords)
                    tagged_images.append(image_path)
        elif method == 'IPTC':
            for filename in os.listdir(folder_path):
                if filename.endswith(('.jpg', '.jpeg', '.png')):
                    image_path = os.path.join(folder_path, filename)
                    add_keywords_to_image_iptc(image_path, keywords)
                    tagged_images.append(image_path)
        elif method == 'All':
            for m in ['XMP', 'EXIF', 'IPTC']:
                for filename in os.listdir(folder_path):
                    if filename.endswith(('.jpg', '.jpeg', '.png')):
                        image_path = os.path.join(folder_path, filename)
                        if m == 'XMP':
                            add_keyword_xmp(image_path, keywords)
                        elif m == 'EXIF':
                            add_keywords_to_image_exif(image_path, keywords)
                        elif m == 'IPTC':
                            add_keywords_to_image_iptc(image_path, keywords)
                        tagged_images.append(image_path)

        else:
            print("Выбран неверный метод.")

        # Создание zip-папки и добавление в нее тегированных изображений
        with ZipFile("tagged_images.zip", "w") as zipf:
            for image_path in tagged_images:
                zipf.write(image_path, os.path.basename(image_path))
        print("Тегированные изображения добавлены в zip-папку.")

    except Exception as e:
        print(f"Ошибка: {e}")

# Интерактивный интерфейс для тегирования изображений
def interactive_tagging():
    folder_text = Text(value='', description='Путь к папке:')
    method_dropdown = Dropdown(options=['All', 'XMP', 'EXIF', 'IPTC'], description='Метод:')

    keyword_text = Text(value='', description='Ключевые слова:')
    button = Button(description="Добавить ключевые слова")

    def on_button_clicked(b):
        folder_path = folder_text.value
        method = method_dropdown.value
        keywords = keyword_text.value.split(',')
        tag_images(folder_path, method, keywords)

    button.on_click(on_button_clicked)

    display(folder_text)
    display(method_dropdown)
    display(keyword_text)
    display(button)

# Запуск интерактивного интерфейса
interactive_tagging()
